<a href="https://colab.research.google.com/github/chiamaka249/IgboNER/blob/main/Copy_of_igbobert4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -c https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
!wget -c https://raw.githubusercontent.com/chiamaka249/lacuna_pos_ner/main/language_corpus/ibo/ibo.txt

--2021-12-08 14:02:15--  https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip [following]
--2021-12-08 14:02:15--  https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7604282 (7.3M) [application/zip]
Saving to: ‘text.zip’

text.zip            100%[===================>]   7.25M  --.-KB/s    in 0.06s   

2021-12-08 14:02:16 (113 MB/s) - ‘text.zip’ saved [7604282/7604282]

--2021-12-08 14:02:16-- 

In [2]:
import zipfile
import os


def unzip(zipfilename):
  try:
    with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
      zip_ref.extractall(zipfilename[:-4])
      return f"'{zipfilename}' unzipped!"
  except FileNotFoundError:
    print(f"Cannot find '{zipfilename}' file")

unzip("text.zip")
!rm text.zip

In [3]:
#copies the file "ibo.txt" to into the folder "text"
import shutil
newPath = shutil.copy('/content/ibo.txt', '/content/text')

In [4]:
# import os
#import shutil
dir_name = "/content/text"
text=""
for fname in os.listdir(dir_name):
  fname = os.path.join(dir_name, fname)
  with open(fname, "r", encoding="utf8") as datafile:
    text = text+"\n"+datafile.read()

with open("data.txt", "w", encoding="utf8") as datafile:
  datafile.write(text)

shutil.rmtree("text")

In [8]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7szm30_z
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-7szm30_z
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
tokenizers                    0.10.3
transformers                  4.13.0.dev0


In [9]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 25.6 s, sys: 1.64 s, total: 27.3 s
Wall time: 7.56 s


In [10]:
!mkdir igbo_bert4
tokenizer.save_model("igbo_bert4")

mkdir: cannot create directory ‘igbo_bert4’: File exists


['igbo_bert4/vocab.json', 'igbo_bert4/merges.txt']

In [11]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./igbo_bert4/vocab.json",
    "./igbo_bert4/merges.txt",
)

In [12]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [13]:
tokenizer.encode("Simone gara ụka ụnyahụ guọ egwu ma ga-kwa taa.").tokens

['<s>',
 'Simone',
 'Ġgara',
 'Ġá»¥ka',
 'Ġá»¥nyahá»¥',
 'Ġgu',
 'á»į',
 'Ġegwu',
 'Ġma',
 'Ġga',
 '-',
 'kwa',
 'Ġtaa',
 '.',
 '</s>']

In [14]:
# Check that we have a GPU
!nvidia-smi

Wed Dec  8 14:13:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [16]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
#from google.colab import files
#files.upload()

In [17]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./igbo_bert4", max_len=512)

In [18]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [19]:
model.num_parameters()
# => 83 million parameters

83504416

In [20]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "/content/data.txt",
    block_size = 128
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 40.8 s, sys: 1.51 s, total: 42.3 s
Wall time: 18.5 s


In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./igbo_bert4",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [23]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 391448
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 30585
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,6.385900
1000,5.310900
1500,4.888700
2000,4.538400
2500,4.315800
3000,4.146700
3500,3.995300
4000,3.867800
4500,3.781600
5000,3.704200


Saving model checkpoint to ./igbo_bert4/checkpoint-10000
Configuration saved in ./igbo_bert4/checkpoint-10000/config.json
Model weights saved in ./igbo_bert4/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./igbo_bert4/checkpoint-20000
Configuration saved in ./igbo_bert4/checkpoint-20000/config.json
Model weights saved in ./igbo_bert4/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to ./igbo_bert4/checkpoint-30000
Configuration saved in ./igbo_bert4/checkpoint-30000/config.json
Model weights saved in ./igbo_bert4/checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [igbo_bert4/checkpoint-10000] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 3h 8min 55s, sys: 53min 32s, total: 4h 2min 28s
Wall time: 4h 1min 3s


TrainOutput(global_step=30585, training_loss=3.1600428904430022, metrics={'train_runtime': 14463.6839, 'train_samples_per_second': 135.321, 'train_steps_per_second': 2.115, 'total_flos': 3.363038734328832e+16, 'train_loss': 3.1600428904430022, 'epoch': 5.0})

In [24]:
trainer.save_model("./igbo_bert4")

Saving model checkpoint to ./igbo_bert4
Configuration saved in ./igbo_bert4/config.json
Model weights saved in ./igbo_bert4/pytorch_model.bin


In [25]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./igbo_bert4",
    tokenizer="./igbo_bert4"
)

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dro

In [26]:
# The sun <mask>.
# =>

fill_mask("Abụ m Maazị <mask>.") #= okafor/Ọkafọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.013978319242596626,
  'sequence': 'Abụ m Maazị Ọkafọ.',
  'token': 5775,
  'token_str': ' Ọkafọ'},
 {'score': 0.010930763557553291,
  'sequence': 'Abụ m Maazị O.',
  'token': 378,
  'token_str': ' O'},
 {'score': 0.01065503153949976,
  'sequence': 'Abụ m Maazị Obianọ.',
  'token': 1566,
  'token_str': ' Obianọ'},
 {'score': 0.00977820809930563,
  'sequence': 'Abụ m Maazị M.',
  'token': 398,
  'token_str': ' M'},
 {'score': 0.0066724903881549835,
  'sequence': 'Abụ m Maazị E.',
  'token': 426,
  'token_str': ' E'}]

In [27]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.12062991410493851,
  'sequence': 'Nwaanyị na ya ji na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.04657674953341484,
  'sequence': 'Nwaanyị na nwunye ji na akara.',
  'token': 740,
  'token_str': ' nwunye'},
 {'score': 0.023468855768442154,
  'sequence': 'Nwaanyị na nna ji na akara.',
  'token': 723,
  'token_str': ' nna'},
 {'score': 0.02191004902124405,
  'sequence': 'Nwaanyị na ha ji na akara.',
  'token': 297,
  'token_str': ' ha'},
 {'score': 0.021505558863282204,
  'sequence': 'Nwaanyị na nwaanyị ji na akara.',
  'token': 629,
  'token_str': ' nwaanyị'}]

In [28]:
# The sun <mask>.
# =>

fill_mask("Chineke ga- ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.3927464187145233,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọjọọ.',
  'token': 726,
  'token_str': ' ọjọọ'},
 {'score': 0.13551250100135803,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọma.',
  'token': 503,
  'token_str': ' ọma'},
 {'score': 0.06629031151533127,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe a.',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.06092679873108864,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe niile.',
  'token': 434,
  'token_str': ' niile'},
 {'score': 0.031389206647872925,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ike.',
  'token': 363,
  'token_str': ' ike'}]

In [36]:
fill_mask("ọba akwụkwọ Ọkammụta Kenneth Dike dị <mask>.") #n'Awka

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.14023567736148834,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị iche.',
  'token': 447,
  'token_str': ' iche'},
 {'score': 0.07257493585348129,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị icheiche.',
  'token': 3871,
  'token_str': ' icheiche'},
 {'score': 0.07037945091724396,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị nso.',
  'token': 606,
  'token_str': ' nso'},
 {'score': 0.042156368494033813,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị ugbua.',
  'token': 1350,
  'token_str': ' ugbua'},
 {'score': 0.04174605384469032,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị egwu.',
  'token': 703,
  'token_str': ' egwu'}]

In [30]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri <mask> na akara.") #= ji
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.13028940558433533,
  'sequence': 'Nwaanyị na eri ya na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.05488795042037964,
  'sequence': 'Nwaanyị na eri nri na akara.',
  'token': 885,
  'token_str': ' nri'},
 {'score': 0.02480449341237545,
  'sequence': 'Nwaanyị na eri ihe na akara.',
  'token': 304,
  'token_str': ' ihe'},
 {'score': 0.019163204357028008,
  'sequence': 'Nwaanyị na eri ọkụ na akara.',
  'token': 707,
  'token_str': ' ọkụ'},
 {'score': 0.017493773251771927,
  'sequence': 'Nwaanyị na eri m na akara.',
  'token': 268,
  'token_str': ' m'}]

In [31]:
# The sun <mask>.
# =>

fill_mask("Ọ bụ <mask>a ka a na- arịa .") #= mmadụ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.22005508840084076,
  'sequence': 'Ọ bụ-a ka a na- arịa.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.019461117684841156,
  'sequence': 'Ọ bụ Nwannaa ka a na- arịa.',
  'token': 1546,
  'token_str': ' Nwanna'},
 {'score': 0.008462444879114628,
  'sequence': 'Ọ bụ ‘a ka a na- arịa.',
  'token': 531,
  'token_str': ' ‘'},
 {'score': 0.0064787482842803,
  'sequence': 'Ọ bụ yaa ka a na- arịa.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.006270089186728001,
  'sequence': 'Ọ bụ ụmụa ka a na- arịa.',
  'token': 428,
  'token_str': ' ụmụ'}]

In [32]:
import shutil

In [33]:
shutil.make_archive("/content/igbo_bert4", 'zip', "igbo_bert4")

'/content/igbo_bert4.zip'

In [34]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [35]:
model_save_name = '/igbo_bert4.zip'
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model.state_dict(), path)

In [ ]:
from google.colab import files
files.download("/content/igbo_bert4.zip")

FileNotFoundError: ignored